# Data Cleaning Dati sul Google Play Store

Librerie

In [44]:
import re
import numpy as np
import pandas as pd
from IPython.display import Markdown

In [64]:
df=pd.read_csv('DATA/raw/data_raw_googleplaystore.csv')

### Rimozione App Duplicate

In [46]:
print(f'Rimozione dei {len(df.App[df.App.duplicated()])} valori duplicati nel campo App.')
df.drop_duplicates(subset='App', inplace=True)

Rimozione dei 1181 valori duplicati nel campo App.


### Trasformazione delle Colonne ['Reviews', 'Size', 'Installs', 'Type', 'Price', 'Current Ver',   'Android Ver']

In [47]:
# transforms to numeric
df['Installs'] = df['Installs'].apply(lambda x: re.sub('[+,]', '', x))
df['Installs'] = df['Installs'].apply(lambda x: x.replace('Free', '-1').replace('Paid', '-1'))
df['Installs'] = pd.to_numeric(df['Installs'])

# transforms to numeric
df['Size'] = df['Size'].apply(lambda x: str(x).replace('Varies with device', '-1'))
df['Size'] = df['Size'].apply(lambda x: re.sub('[M,]', '', x))
df['Size'] = df['Size'].apply(lambda x: (float(str(x).replace('k', '')) / 1000) if 'k' in str(x) else x)
df['Size'] = pd.to_numeric(df['Size'], errors = 'coerce')

# transforms to numeric
df['Price'] = df['Price'].apply(lambda x: str(x).replace('$', '') if '$' in str(x) else str(x))
df['Price']=df['Price'].replace('Free',0)
df['Price'] = pd.to_numeric(df['Price'], errors = 'coerce')

# transforms to numeric
df['Android Ver'] = df['Android Ver'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
df['Android Ver'] = pd.to_numeric(df['Android Ver'], errors = 'coerce')

# transforms to numeric
df['Current Ver'] = df['Current Ver'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
df['Current Ver'] = pd.to_numeric(df['Current Ver'], errors = 'coerce')

# transforms to numeric
df['Reviews'] = df['Reviews'].apply(lambda x: x.replace('3.0M', '3000000'))
df['Reviews'] = pd.to_numeric(df['Reviews'], errors = 'coerce')

### Sanity Checks

#### Installs

In [48]:
print(f'Rimozione delle {len(df.App[df.Installs==0])} App che non sono mai state installate.')
df.drop(df[df.Installs==0].index,inplace=True)

Rimozione delle 15 App che non sono mai state installate.


#### Rating

In [49]:
print(f'Rimozione delle App con valori del rating non conformi, che è su una scala da 1 a 5.')
df.drop(df.loc[(df['Rating']<1) | (df['Rating']>5)].index, inplace=True)


Rimozione delle App con valori del rating non conformi, che è su una scala da 1 a 5.


#### Price

In [50]:
print(f'Rimozione delle {len(df[df.Price>250])} App con prezzo superiore a 250 USD.')
df.drop(df.loc[df.Price>250].index, inplace=True)

Rimozione delle 15 App con prezzo superiore a 250 USD.


#### Category

In [51]:
print('Analisi delle etichette e dei valori delle varie categorie, sosituzione di valori anomali.')
df.Category.value_counts().to_frame()

Analisi delle etichette e dei valori delle varie categorie, sosituzione di valori anomali.


,Category
FAMILY,1825
GAME,959
TOOLS,827
BUSINESS,419
MEDICAL,394
PERSONALIZATION,375
PRODUCTIVITY,373
LIFESTYLE,363
FINANCE,337
SPORTS,325


In [52]:
df['Category'].replace({'1.9': 'PHOTOGRAPHY'}, inplace = True)

### Genres

In [53]:
print('Pulizia e uniformazione del campo Genres.')

Pulizia e uniformazione del campo Genres.


In [54]:
# check all the possible genres
gen = df['Genres'].values
# gen = [el.replace('&', 'and').replace(' ', '_')for el in gen]
gen = [el.replace(' ', '' ) for el in gen]
g = set()
for el in gen:
  el = el.lower()
  g = g.union(set(el.split(';')))

# g1 is the set of all possbile genres
g1 = set()
for el in g:
  g1 = g1.union(set(el.split('&')))

print(g1)

{'photography', 'beauty', 'tools', 'casual', 'arcade', 'adventure', 'libraries', 'fitness', 'business', 'events', 'puzzle', 'art', 'communication', 'magazines', 'roleplaying', 'home', 'social', 'sports', 'food', 'comics', 'audio', 'action', 'demo', 'educational', 'health', 'racing', 'education', 'productivity', 'weather', 'parenting', 'entertainment', 'lifestyle', 'music', 'pretendplay', 'books', 'card', 'simulation', 'design', 'drink', 'dating', 'editors', 'maps', 'local', 'casino', 'navigation', 'shopping', 'vehicles', 'travel', 'braingames', 'trivia', 'reference', 'board', 'strategy', 'news', 'personalization', 'finance', 'video', 'videoplayers', 'word', 'auto', 'medical', 'creativity', 'house'}


In [55]:
def genres_cleaner(stringa):
  stringa = stringa.lower()

  rep_dict = {'&': ';',
              ' ': '',
              'house': 'home',
              'educational': 'education',
              'february11,2018': ''}

  for key, val in rep_dict.items():
    stringa = stringa.replace(key, val)
  return stringa

df['Genres'] = df['Genres'].apply(lambda x: genres_cleaner(x))

### Last Updated

In [56]:
print('Trasformazione del formato del campo Last Updated.')
# last updated to datetime
df['Last Updated'].replace({'1.0.19': np.nan}, inplace = True)
df['Last Updated'] = pd.to_datetime(df['Last Updated'], infer_datetime_format = True)

Trasformazione del formato del campo Last Updated.


### Content Rating

In [57]:
print(df['Content Rating'].value_counts().to_frame())

                 Content Rating
Everyone                   7877
Teen                       1033
Mature 17+                  393
Everyone 10+                321
Adults only 18+               3
Unrated                       2


In [58]:
df[df['Content Rating'] == 'Mature 17+'].head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
129,"Secrets of beauty, youth and health",BEAUTY,NaN,77,2.9,10000,Free,0.0,Mature 17+,beauty,2017-08-08,20.0,23.0
133,Dresses Ideas & Fashions +3000,BEAUTY,4.5,473,8.2,100000,Free,0.0,Mature 17+,beauty,2017-03-01,1020.0,16.0
134,"Lady adviser (beauty, health)",BEAUTY,NaN,30,9.9,10000,Free,0.0,Mature 17+,beauty,2018-01-24,30.0,30.0
166,Read books online,BOOKS_AND_REFERENCE,4.1,91615,23.0,5000000,Free,0.0,Mature 17+,books;reference,2018-06-29,312.0,41.0
170,Flybook,BOOKS_AND_REFERENCE,3.9,1778,4.9,500000,Free,0.0,Mature 17+,books;reference,2014-10-16,103.0,21.0


In [59]:
df[df['Content Rating'] == 'Adults only 18+'].head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
298,Manga Master - Best manga & comic reader,COMICS,4.6,24005,4.9,500000,Free,0.0,Adults only 18+,comics,2018-07-04,1170.0,41.0
3043,DraftKings - Daily Fantasy Sports,SPORTS,4.5,50017,41.0,1000000,Free,0.0,Adults only 18+,sports,2018-07-24,321324.0,44.0
6424,Manga Books,COMICS,3.8,7326,-1.0,500000,Free,0.0,Adults only 18+,comics,2018-08-03,NaN,NaN


In [60]:
df['Content Rating'].replace({'Unrated': 'Everyone',
                              'Adults only 18+': 'Mature 17+',
                              'Everyone 10+': 'Everyone'}, inplace = True)

In [61]:
df['Content Rating'].value_counts().to_frame()

,Content Rating
Everyone,8200
Teen,1033
Mature 17+,396


### Trattamento e Imputazione dei Missing Values: campi Type, Content Rating, Android Ver, Current Ver, Size

In [62]:
# fillna
df['Type'].fillna('Free', inplace = True)

# fillna
df['Content Rating'].fillna('Everyone', inplace = True)

# impute nan with most frequent strategy
cols_imp = ['Android Ver', 'Current Ver', 'Size']
simple_imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[cols_imp] = simple_imp.fit_transform(df[cols_imp])

In [63]:
#df.to_csv('DATA/pre-processed/clean_app_data.csv',index=False)